# Final Exam
So, after a lot of messing with things, I realized that this needed to be reproducible (something a smarter person than myself would have known from the start). So I switched over to Jupyter Notebooks. I hope this is acceptable.

## The idea:
I figured we could identify areas where low ranking (via number of stars) could improve upon in order to maximize their star increase. E.g, a coffeshop without wifi might want to get wifi. As it turns out, getting paid wifi is not a good idea for such a coffee shop (go big or go home, I guess).

## Areas for improvement:
1. Obviously we would do this with every category (possibly once per year or so).

2. Add natural language processing (possibly via nltk available at nltk.org) to classify customer types from the language used in reviews, and suggest amenities that these customers tend to like.

In [167]:
import numpy as np
import json
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
from pymongo import MongoClient

In [6]:
db = MongoClient().fall_db_final

In [7]:
business = db.academic_business

In [105]:
def getDF(table, predicate, preprocessor=lambda x: x, projection=None):
    tmp = table.find(predicate, projection)
    maxInd = tmp.count()
    print("Found {r} records matching this predicate.".format(r=maxInd))
    retVal = []
    for i in range(maxInd):
        retVal.append(preprocessor(tmp[i]))
    del tmp
    del maxInd
    retVal = pd.DataFrame.from_records(retVal)
    return retVal

In [104]:
def getWifi(rec):
    if 'Wi-Fi' in rec['attributes']:
        rec['wifi'] = rec['attributes']['Wi-Fi']
    else:
        rec['wifi'] = 'Unknown'
    rec.pop('attributes', None)
    return rec

In [107]:
bars = getDF(business,
             {'categories': {'$in': ['Bars']}},
             projection={'attributes.Wi-Fi': 1,
                         'city': 1,
                         'name': 1,
                         'stars': 1},
             preprocessor=getWifi
            )

Found 4727 records matching this predicate.


In [109]:
bars[bars['wifi'] == 'free'].mean()

stars    3.541452
dtype: float64

In [110]:
bars[bars['wifi'] == 'no'].mean()

stars    3.49045
dtype: float64

In [94]:
bars[bars['wifi'] == 'Unknown'].mean()

stars    3.550434
dtype: float64

In [95]:
bars[bars['wifi'] == 'paid'].mean()

stars    3.473684
dtype: float64

In [112]:
bars_lm = ols('stars ~ C(wifi)', data=bars).fit()

In [113]:
print(sm.stats.anova_lm(bars_lm, typ=2))

               sum_sq      df         F    PR(>F)
C(wifi)      2.128350     3.0  1.633234  0.179445
Residual  2051.593355  4723.0       NaN       NaN


In [114]:
food = getDF(business,
              {'categories': {'$in': ['Food']}},
              projection={'attributes.Wi-Fi': 1,
                          'city': 1,
                          'name': 1,
                          'stars': 1},
              preprocessor=getWifi
             )

Found 10143 records matching this predicate.


In [116]:
food_lm = ols('stars ~ C(wifi)', data=food).fit()
print(sm.stats.anova_lm(food_lm, typ=2))

               sum_sq       df         F    PR(>F)
C(wifi)      8.706789      3.0  4.562756  0.003375
Residual  6449.181902  10139.0       NaN       NaN


In [118]:
coffee = getDF(business,
               {'categories': {'$in': ['Coffee & Tea']}},
               projection={'attributes.Wi-Fi': 1,
                           'city': 1,
                           'name': 1,
                           'stars': 1},
               preprocessor=getWifi
              )

Found 2399 records matching this predicate.


In [119]:
coffee_lm = ols('stars ~ C(wifi)', data=coffee).fit()
print(sm.stats.anova_lm(coffee_lm, typ=2))

               sum_sq      df         F    PR(>F)
C(wifi)      8.614933     3.0  5.058178  0.001709
Residual  1359.696655  2395.0       NaN       NaN


In [131]:
coffee_lm.pvalues

Intercept          0.000000
C(wifi)[T.free]    0.225360
C(wifi)[T.no]      0.243320
C(wifi)[T.paid]    0.007715
dtype: float64

In [132]:
coffee.groupby(by='wifi').mean()

,stars
wifi,
Unknown,3.764012
free,3.709521
no,3.829114
paid,3.250000


So we can see that it's pretty bad to charge for wifi at your coffeeshop (the low p-value associated to 'paid' wifi suggests that this is not just spurious).

In [139]:
lowRankCoffee = coffee[coffee['stars'] < 3]

In [145]:
def getAllAttrs(rec):
    rec = dict(rec, **rec['attributes'])
    rec.pop('attributes', None)
    return rec

In [146]:
coffee_full = getDF(business,
                    {'categories': {'$in': ['Coffee & Tea']}},
                    projection={'attributes': 1,
                                'city': 1,
                                'name': 1,
                                'stars': 1},
                    preprocessor=getAllAttrs
                   )

Found 2399 records matching this predicate.


In [151]:
coffee_full

,Accepts Credit Cards,Accepts Insurance,Ages Allowed,Alcohol,Ambience,Attire,BYOB,BYOB/Corkage,By Appointment Only,Caters,...,Smoking,Take-out,Takes Reservations,Waiter Service,Wheelchair Accessible,Wi-Fi,_id,city,name,stars
0,True,NaN,NaN,none,NaN,NaN,NaN,NaN,False,NaN,...,NaN,True,NaN,NaN,True,free,5848a79739a36819a9b309e4,Homestead,Starbucks,3.5
1,True,NaN,NaN,none,"{'divey': False, 'romantic': False, 'hipster':...",casual,NaN,NaN,NaN,False,...,NaN,True,False,False,True,free,5848a79739a36819a9b30a55,Pittsburgh,Tazza D'oro Cafe & Espresso Bar,4.5
2,True,NaN,NaN,none,"{'divey': False, 'romantic': False, 'hipster':...",casual,NaN,yes_corkage,NaN,True,...,NaN,True,False,True,True,free,5848a79739a36819a9b30a79,Pittsburgh,Quiet Storm Vegetarian & Vegan Cafe,4.0
3,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,no,5848a79739a36819a9b30abc,Pittsburgh,Cool Beans Coffee,5.0
4,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,True,free,5848a79739a36819a9b30b16,Pittsburgh,Katerbean,1.5
5,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,NaN,True,free,5848a79739a36819a9b30b37,Pittsburgh,Starbucks,4.5
6,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,NaN,NaN,free,5848a79739a36819a9b30b4b,Pittsburgh,Starbucks,3.5
7,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,NaN,True,no,5848a79739a36819a9b30b64,Pittsburgh,Nicholas Coffee,4.0
8,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,NaN,True,no,5848a79739a36819a9b30b79,Pittsburgh,La Prima Espresso Co,4.5
9,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,no,5848a79739a36819a9b30b7d,Pittsburgh,Fifth Avenue Beanery,3.5


In [176]:
tmp = coffee_full[~ pd.isnull(coffee_full['Ambience'])]['Ambience']
tmp

1       {'divey': False, 'romantic': False, 'hipster':...
2       {'divey': False, 'romantic': False, 'hipster':...
12      {'divey': False, 'romantic': False, 'hipster':...
24      {'divey': True, 'romantic': False, 'hipster': ...
31      {'divey': False, 'romantic': False, 'hipster':...
33      {'divey': False, 'romantic': False, 'hipster':...
42      {'divey': False, 'romantic': False, 'hipster':...
43      {'divey': False, 'romantic': False, 'hipster':...
52      {'divey': False, 'romantic': False, 'hipster':...
54      {'divey': True, 'romantic': False, 'hipster': ...
76      {'divey': False, 'romantic': False, 'hipster':...
78      {'divey': False, 'romantic': False, 'hipster':...
98      {'divey': False, 'romantic': False, 'hipster':...
103     {'divey': False, 'romantic': False, 'hipster':...
105     {'divey': False, 'romantic': False, 'hipster':...
112     {'divey': True, 'romantic': False, 'hipster': ...
115     {'divey': False, 'romantic': False, 'hipster':...
118     {'dive

In [182]:
tmp.irow(0)['casual']

/home/amniskin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: irow(i) is deprecated. Please use .iloc[i] or .iat[i]
  if __name__ == '__main__':


True